In [1]:
!pip install transformers

In [2]:
!pip install tensorflow transformers keras

In [3]:
!pip install langdetect

In [4]:
!pip install wordcloud

In [5]:
!pip install geopy

In [6]:
!pip install geopandas

In [7]:
# Importing libraries
import math

import numpy as np
import pandas as pd
import torch

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from tqdm.auto import tqdm

import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import spacy
import random

from langdetect import detect
from wordcloud import WordCloud

from keras.layers import Input, Dense
from keras.models import Model

from geopy.geocoders import Nominatim
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import BertForSequenceClassification, BertModel, BertTokenizer, TFBertModel
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import DataLoader, Dataset

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords as stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
df = pd.read_csv('merged.csv')

In [9]:
df

,id,text,user_id,timestamp,cluster_id,lat,lng,language_label,year,month,day,hour,minute,second
0,0,wow ’ know homies coming . lit .,5.853807e+07,2021-08-02 01:25:46,2,34.020789,-118.411907,0,2021.0,8.0,2.0,1.0,25.0,46.0
1,1,foggy nights peaceful calming chill vibe,1.032647e+18,2021-08-28 08:47:36,2,34.020789,-118.411907,0,2021.0,8.0,28.0,8.0,47.0,36.0
2,2,khamisvideobomb,2.781174e+07,2021-08-13 15:49:15,2,34.020789,-118.411907,24,2021.0,8.0,13.0,15.0,49.0,15.0
3,3,anyone tryna hit gym 4 hmu,5.490040e+08,2021-08-31 22:41:20,2,34.020789,-118.411907,24,2021.0,8.0,31.0,22.0,41.0,20.0
4,4,kidding intro crazy .,2.963951e+07,2021-08-06 03:44:29,2,34.020789,-118.411907,4,2021.0,8.0,6.0,3.0,44.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604201,604201,students must dressed appropriately day arrive...,1.990241e+08,2021-08-12 11:08:07,2996,29.668301,-95.158936,0,2021.0,8.0,12.0,11.0,8.0,7.0
604202,604202,actually hate working lunch shift crew job,1.382980e+18,2021-08-04 15:35:10,2996,29.668301,-95.158936,0,2021.0,8.0,4.0,15.0,35.0,10.0
604203,604203,’ reading thisyo mama,1.039173e+18,2021-08-24 00:49:46,2996,29.668301,-95.158936,24,2021.0,8.0,24.0,0.0,49.0,46.0
604204,604204,mean secondddd,1.046619e+18,2021-08-02 17:02:13,2996,29.668301,-95.158936,0,2021.0,8.0,2.0,17.0,2.0,13.0


In [10]:
from sklearn.cluster import KMeans
from math import radians, cos, sin, asin, sqrt

def haversine(lat1, lon1, lat2, lon2):
    R = 6372.8  # Earth radius in kilometers
 
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
 
    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))
 
    return R * c


In [11]:
coordinates = df[['lat', 'lng']].to_numpy()


In [12]:
kmeans = KMeans(n_clusters=100, random_state=0).fit(coordinates)


In [13]:
df['cluster_label'] = kmeans.labels_


In [14]:
for i in range(100): # 100 is the number of clusters
    cluster_center = kmeans.cluster_centers_[i]
    cluster_lat, cluster_lng = cluster_center[0], cluster_center[1]
    df.loc[df['cluster_label'] == i, 'distance_from_center'] = \
        df.loc[df['cluster_label'] == i]\
        .apply(lambda row: haversine(row['lat'], row['lng'], cluster_lat, cluster_lng), axis=1)


In [15]:
df

,id,text,user_id,timestamp,cluster_id,lat,lng,language_label,year,month,day,hour,minute,second,cluster_label,distance_from_center
0,0,wow ’ know homies coming . lit .,5.853807e+07,2021-08-02 01:25:46,2,34.020789,-118.411907,0,2021.0,8.0,2.0,1.0,25.0,46.0,1,13.970579
1,1,foggy nights peaceful calming chill vibe,1.032647e+18,2021-08-28 08:47:36,2,34.020789,-118.411907,0,2021.0,8.0,28.0,8.0,47.0,36.0,1,13.970579
2,2,khamisvideobomb,2.781174e+07,2021-08-13 15:49:15,2,34.020789,-118.411907,24,2021.0,8.0,13.0,15.0,49.0,15.0,1,13.970579
3,3,anyone tryna hit gym 4 hmu,5.490040e+08,2021-08-31 22:41:20,2,34.020789,-118.411907,24,2021.0,8.0,31.0,22.0,41.0,20.0,1,13.970579
4,4,kidding intro crazy .,2.963951e+07,2021-08-06 03:44:29,2,34.020789,-118.411907,4,2021.0,8.0,6.0,3.0,44.0,29.0,1,13.970579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604201,604201,students must dressed appropriately day arrive...,1.990241e+08,2021-08-12 11:08:07,2996,29.668301,-95.158936,0,2021.0,8.0,12.0,11.0,8.0,7.0,4,33.786445
604202,604202,actually hate working lunch shift crew job,1.382980e+18,2021-08-04 15:35:10,2996,29.668301,-95.158936,0,2021.0,8.0,4.0,15.0,35.0,10.0,4,33.786445
604203,604203,’ reading thisyo mama,1.039173e+18,2021-08-24 00:49:46,2996,29.668301,-95.158936,24,2021.0,8.0,24.0,0.0,49.0,46.0,4,33.786445
604204,604204,mean secondddd,1.046619e+18,2021-08-02 17:02:13,2996,29.668301,-95.158936,0,2021.0,8.0,2.0,17.0,2.0,13.0,4,33.786445


In [16]:
df.to_csv('merged_cluster100.csv', index=False)
